# LA Dodgers toplines
> This notebook extracts key statistics from the project's processed tables for display in a dashboard.

---

#### Import Python tools and Jupyter config

In [1]:
import os
import boto3
import pandas as pd
import jupyter_black
import altair as alt
from io import BytesIO

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

## Read

#### Standings

In [ ]:
history = pd.read_parquet(
    "https://stilesdata.com/dodgers/data/standings/archive/dodgers_standings_1958_2023.parquet"
)

In [ ]:
history[["r", "ra", "attendance", "gm", "rank"]] = history[
    ["r", "ra", "attendance", "gm", "rank"]
].astype(int)

In [31]:
history.to_parquet(
    "../data/standings/archive/dodgers_standings_1958_2023.parquet", index=False
)
history.to_json(
    "../data/standings/archive/dodgers_standings_1958_2023.json",
    indent=4,
    orient="records",
)
history.to_csv("../data/standings/archive/dodgers_standings_1958_2023.csv", index=False)

In [ ]:
standings = pd.read_parquet(
    "https://stilesdata.com/dodgers/data/standings/dodgers_standings_1958_present.parquet"
).query("year == '2024'")
standings_past = pd.read_parquet(
    "https://stilesdata.com/dodgers/data/standings/dodgers_standings_1958_present.parquet"
).query("year != '2024'")
standings_now = standings.query("game_date == game_date.max()").copy()

In [25]:
standings_now.loc[standings_now.result == "L", "result_clean"] = "loss"
standings_now.loc[standings_now.result == "W", "result_clean"] = "win"

In [26]:
standings.head()

,gm,game_date,home_away,opp,result,r,ra,record,rank,gb,time,time_minutes,day_night,attendance,year,wins,losses,win_pct,game_day
0,26,2024-04-24,away,WSN,W,11,2,15-11,1,1.5,2:40:00,160,N,26298,2024,15,11,0.58,Wednesday
1,25,2024-04-23,away,WSN,W,4,1,14-11,1,1.5,3:10:00,190,N,27806,2024,14,11,0.56,Tuesday
2,24,2024-04-21,home,NYM,W,10,0,13-11,1,1.0,2:18:00,138,D,49287,2024,13,11,0.54,Sunday
3,23,2024-04-20,home,NYM,L,4,6,12-11,1,1.0,3:20:00,200,D,45373,2024,12,11,0.52,Saturday
4,22,2024-04-19,home,NYM,L,4,9,12-10,1,1.0,3:03:00,183,N,44783,2024,12,10,0.55,Friday


In [11]:
batting = pd.read_parquet(
    "https://stilesdata.com/dodgers/data/batting/dodgers_team_batting_1958_present.parquet"
)

In [12]:
batting_past = batting.query("season != '2024'").copy()
batting_now = batting.query("season == '2024'").copy()

---

## Key statistics

#### 1. Current season record (Wins-Losses)
> Provides an immediate understanding of the team's overall performance for the season.

In [7]:
games = standings_now["gm"].loc[0]
wins = standings_now["wins"].loc[0]
losses = standings_now["losses"].loc[0]

In [8]:
record = standings_now["record"].loc[0]

#### 2. Win percentage
> Allows for normalization of success to compare across different seasons or different numbers of games played.

In [9]:
win_pct = int(standings_now["win_pct"].loc[0] * 100)
win_pct_decade_thispoint = int(
    standings_past.query(f"gm == {games}").head(10)["win_pct"].mean().round(2) * 100
)

#### 3. Run differential
> A positive run differential generally correlates with a stronger team performance and is predictive of future success.

In [10]:
runs = standings["r"].sum()
runs_against = standings["ra"].sum()

In [11]:
run_diff = runs - runs_against

#### 4. Home runs and home runs per game
> Reflects the team's power-hitting capabilities, significant for scoring strategies.

In [12]:
batting_past["hr_game"] = (
    batting_past["hr"].astype(int) / batting_past["g"].astype(int)
).round(2)

In [13]:
home_runs = int(batting_now["hr"].sum())
home_runs_game = (home_runs / games).round(2)
home_runs_game_last = batting_past.query('season == "2023"')["hr_game"].iloc[0]

In [14]:
games_decade = batting_past.head(10)["g"].astype(int).sum()
home_runs_decade = batting_past.head(10)["hr"].astype(int).sum()

In [15]:
home_runs_game_decade = (home_runs_decade / games_decade).round(2)

#### 5. Earned run average (ERA)
> A key measure of pitching staff effectiveness, with a lower ERA indicating better performance.

#### Batting average and on
> Summarizes players' strength in getting on base — and hopefully scoring runs.

In [16]:
batting_average = batting_now["ba"].iloc[0]

In [17]:
batting_average_decade = (
    batting_past.head(10)["ba"]
    .astype(float)
    .mean()
    .round(3)
    .astype(str)
    .replace("0.", ".")
)

#### 7. Stolen bases
> Stolen bases can significantly impact game dynamics and indicate the team's strategic play.

In [18]:
stolen_bases = int(batting_now["sb"].iloc[0])
stolen_bases_game = (stolen_bases / games).round(2)

In [19]:
stolen_decade = batting_past.head(10)["sb"].astype(int).sum()
games_decade = batting_past.head(10)["g"].astype(int).sum()
stolen_bases_decade_game = (stolen_decade / games_decade).round(2)

#### 8. Fielding percentage
> Indicates the team's defensive capabilities, with a higher percentage reflecting better performance.

#### 9. Recent trend (last 10 games)
> Provides insight into the team's current form and momentum, which is essential for assessing changes in performance.

In [20]:
last_10 = standings["result"].head(10)
win_count_trend = last_10[last_10 == "W"].count()
loss_count_trend = last_10[last_10 == "L"].count()

In [21]:
win_loss_trend = f"Recent trend: {win_count_trend} wins, {loss_count_trend} losses"

#### 10. Summary
> Creates one file to import for topline statistics and a narrative summary of the standings now.

In [22]:
summary = f"The Dodgers have played {games} games this season compiling a {record} record — a winning percentage of {win_pct}%. The team's last game was a {standings_now['r'].iloc[0]}-{standings_now['ra'].iloc[0]} {standings_now['home_away'].iloc[0]} {standings_now['result_clean'].iloc[0]} to the {standings_now['opp'].iloc[0]} in front of {'{:,}'.format(standings_now['attendance'].iloc[0])} fans. The team has won {win_count_trend} of its last 10 games."

In [36]:
summary_data = [
    {"stat": "wins", "value": wins, "category": "standings"},
    {"stat": "losses", "value": losses, "category": "standings"},
    {"stat": "record", "value": record, "category": "standings"},
    {"stat": "win_pct", "value": f"{win_pct}%", "category": "standings"},
    {
        "stat": "win_pct_decade_thispoint",
        "value": f"{win_pct_decade_thispoint}%",
        "category": "standings",
    },
    {"stat": "runs", "value": runs, "category": "standings"},
    {"stat": "runs_against", "value": runs_against, "category": "standings"},
    {"stat": "run_differential", "value": run_diff, "category": "standings"},
    {"stat": "home_runs", "value": home_runs, "category": "batting"},
    {"stat": "home_runs_game", "value": home_runs_game, "category": "batting"},
    {
        "stat": "home_runs_game_last",
        "value": home_runs_game_last,
        "category": "batting",
    },
    {
        "stat": "home_runs_game_decade",
        "value": home_runs_game_decade,
        "category": "batting",
    },
    {"stat": "stolen_bases", "value": stolen_bases, "category": "batting"},
    {"stat": "stolen_bases_game", "value": stolen_bases_game, "category": "batting"},
    {
        "stat": "stolen_bases_decade_game",
        "value": stolen_bases_decade_game,
        "category": "batting",
    },
    {"stat": "batting_average", "value": batting_average, "category": "batting"},
    {
        "stat": "batting_average_decade",
        "value": batting_average_decade,
        "category": "batting",
    },
    {"stat": "summary", "value": summary, "category": "standings"},
]

In [ ]:
summary_df = pd.DataFrame(summary_data)

In [38]:
summary_df

,stat,value,category
0,wins,15,standings
1,losses,11,standings
2,record,15-11,standings
3,win_pct,57%,standings
4,win_pct_decade_thispoint,57%,standings
5,runs,139,standings
6,runs_against,112,standings
7,run_differential,27,standings
8,home_runs,30,batting
9,home_runs_game,1.15,batting


In [25]:
summary_df.to_csv("../data/standings/season_summary_latest.csv", index=False)
summary_df.to_json(
    "../data/standings/season_summary_latest.csv", indent=4, orient="records"
)

#### S3

In [27]:
def save_to_s3(df, base_path, s3_bucket, formats=["csv", "json"]):
    """
    Save Pandas DataFrame in specified formats and upload to S3 bucket using environment credentials.

    :param df: DataFrame to save.
    :param base_path: Base file path without format extension.
    :param s3_bucket: S3 bucket name.
    :param formats: List of formats to save -- 'csv', 'json'.
    """
    # Create session using environment variables directly
    session = boto3.Session(
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name="us-west-1",
        profile_name="haekeo",
    )
    s3_resource = session.resource("s3")

    for fmt in formats:
        file_path = f"{base_path}.{fmt}"
        buffer = BytesIO()
        if fmt == "csv":
            df.to_csv(buffer, index=False)
            content_type = "text/csv"
        elif fmt == "json":
            df.to_json(buffer, orient="records", lines=True)
            content_type = "application/json"

        buffer.seek(0)
        s3_resource.Bucket(s3_bucket).put_object(
            Key=file_path, Body=buffer, ContentType=content_type
        )
        print(f"Uploaded {fmt} to {s3_bucket}/{file_path}")


# Save to S3
save_to_s3(
    summary_df,
    "dodgers/data/standings/season_summary_latest",
    "stilesdata.com",
)

Uploaded csv to stilesdata.com/dodgers/data/standings/season_summary_latest.csv
Uploaded json to stilesdata.com/dodgers/data/standings/season_summary_latest.json


In [ ]:
# !jupyter nbconvert --to script --no-prompt --output ../scripts/06-create-toplines-summary 07-create-toplines-summary.ipynb

In [33]:
summary_df.dtypes

stat          object
stat_value    object
category      object
dtype: object